## 5.2 Essential Functionality（主要功能）
接下来介绍pandas中的一些主要功能，这里只介绍一些经常用到的。

## 1 Reindexing（重新索引）
pandas中一个重要的方法是reindex，已实施在创建object的时候遵照一个新的index。如下例：

In [1]:
import pandas as pd

In [2]:
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=['d','b','a','c'])
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

在series上调用reindex能更改index，如果没有对应index的话会引入缺失数据：

In [3]:
obj2 = obj.reindex(['a','b','c','d','e'])
obj2

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

在处理时间序列这样的数据时，我们可能需要在reindexing的时候需要修改值。method选项能做到这一点，比如设定method为ffill:

In [4]:
obj3 = pd.Series(['bule','purple','yellow'], index=[0,4,8])
obj3

0      bule
4    purple
8    yellow
dtype: object

In [5]:
obj3.reindex(range(10), method='ffill')

0      bule
1      bule
2      bule
3      bule
4    purple
5    purple
6    purple
7    purple
8    yellow
9    yellow
dtype: object

对于DataFrame，reindex能更改row index,或column index。reindex the rows:

In [6]:
import numpy as np

In [7]:
frame = pd.DataFrame(np.arange(9).reshape(3, 3),
                     index=['a','c','d'],
                            columns = ['Ohio','Texas','California'])

In [8]:
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [9]:
frame2 = frame.reindex(['a','b','c','d'])
frame2

,Ohio,Texas,California
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


更改columns index:

In [10]:
states = ['Texas','Utah','California']

In [11]:
frame.reindex(columns=states)

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


reindex的参数：



还可以使用loc更简洁的reindex：

In [12]:
frame.loc[['a','b','c','d'],states]

,Texas,Utah,California
a,1.0,NaN,2.0
b,NaN,NaN,NaN
c,4.0,NaN,5.0
d,7.0,NaN,8.0


## 2 Dropping Entries from an Axis (按轴删除记录)
对于series，drop回返回一个新的object，并删去你制定的axis的值：

In [13]:
obj = pd.Series(np.arange(5.), index=['a','b','c','d','e'])
obj

a    0.0
b    1.0
c    2.0
d    3.0
e    4.0
dtype: float64

In [14]:
new_obj = obj.drop('c')
new_obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

In [15]:
obj.drop(['d','c'])

a    0.0
b    1.0
e    4.0
dtype: float64

对于DataFrame，index能按行或列的axis来删除：

In [16]:
data = pd.DataFrame(np.arange(16).reshape(4,4),
                   index=['Ohio','Colorado','Utah','New York'],
                   columns=['one','two','three','four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


行处理：如果a sequence of labels(一个标签序列)来调用drop，会删去row labels(axis 0):

In [17]:
data.drop(['Colorado','Ohio'])

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


列处理：drop列的话，设定axis=1或axis='columns':

In [18]:
data.drop('two', axis =1)

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


In [19]:
data.drop(['two','four'], axis='columns')

,one,three
Ohio,0,2
Colorado,4,6
Utah,8,10
New York,12,14


drop也可以不返回一个新的object，而是直接更改series or dataframe in-place:

In [20]:
obj.drop('c', inplace=True)
obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

## 3 Indexing, Selection, and Filtering(索引，选择，过滤)
series indexing(obj[...]) 相当于numpy的array indexing, 而且除了整数，还可以使用series的index：

In [21]:
obj = pd.Series(np.arange(4.),index=['a','b','c','d'])
obj

a    0.0
b    1.0
c    2.0
d    3.0
dtype: float64

In [22]:
obj['b']

1.0

In [23]:
obj[1]

1.0

In [24]:
obj[2:4]

c    2.0
d    3.0
dtype: float64

In [25]:
obj[['b','a','d']]

b    1.0
a    0.0
d    3.0
dtype: float64

In [26]:
obj[[1,3]]

b    1.0
d    3.0
dtype: float64

In [27]:
obj[obj<2]

a    0.0
b    1.0
dtype: float64

用label来slicing(切片)的时候，和python的切片不一样的在于，会包括尾节点：

In [28]:
obj['b':'c']

b    1.0
c    2.0
dtype: float64

可以直接给选中的label更改值：

In [29]:
obj['b':'c'] = 5
obj

a    0.0
b    5.0
c    5.0
d    3.0
dtype: float64

而对于DataFrame，indexing可以通过一个值或序列，选中一个以上的列：

In [30]:
data = pd.DataFrame(np.arange(16).reshape((4,4)),
                   index=['Ohio','Colorado','Utah','New York'],
                   columns=['one','two','three','four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [31]:
data['two']

Ohio         1
Colorado     5
Utah         9
New York    13
Name: two, dtype: int32

In [32]:
data[['three','one']]

,three,one
Ohio,2,0
Colorado,6,4
Utah,10,8
New York,14,12


dataframe的indexing有一些比较特别的方式。比如通过布尔数组：

In [33]:
data[:3]

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11


In [34]:
data[data['three']>5]

,one,two,three,four
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


行选择的语法格式data[:2]是很方便的。给[]里传入一个list的话，可以选择列。

另一种方法是用boolean dataframe:

In [35]:
data<5

,one,two,three,four
Ohio,True,True,True,True
Colorado,True,False,False,False
Utah,False,False,False,False
New York,False,False,False,False


In [37]:
data[data<5]=0
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


## Selection with loc and iloc(用loc和iloc来选择)
对于label-indexing on rows, 我们介绍特别的索引符，loc and iloc. 这两个方法能通过axis labels(loc)或integer(iloc)，来选择行或列。

一个列子，选中一行多列by label：

In [38]:
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [39]:
data.loc['Colorado',['two','three']]

two      5
three    6
Name: Colorado, dtype: int32

In [40]:
data.loc['Utah',['one','four']]

one      8
four    11
Name: Utah, dtype: int32

同iloc实现相同的效果：

In [41]:
data.iloc[2,[3,0,1]]

four    11
one      8
two      9
Name: Utah, dtype: int32

In [42]:
data.iloc[1,[1,3]]

two     5
four    7
Name: Colorado, dtype: int32

In [44]:
data.iloc[[1,2],[3,0,1]]

,four,one,two
Colorado,7,0,5
Utah,11,8,9


In [47]:
data.iloc[2]

one       8
two       9
three    10
four     11
Name: Utah, dtype: int32

data.iloc[2]表示：这个数据的第二行。

indexing函数也能用于切片，不论是single labels或lists of labels:

In [45]:
data.loc[:'Utah','two']

Ohio        0
Colorado    5
Utah        9
Name: two, dtype: int32

In [46]:
data.loc[:'Colorado','three']

Ohio        0
Colorado    6
Name: three, dtype: int32

有个冒号在前面，表示： 显示到这个“冒号后的位置”的地方，后面的栏就是要列的位置

In [49]:
data.iloc[:, :3][data.three>5]

,one,two,three
Colorado,0,5,6
Utah,8,9,10
New York,12,13,14


In [57]:
data.iloc[:, :4][data.four > -0.1]

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


注意：当设计padnas的时候，作者发现frame[:, col]这样的语法是比较冗长的，因为这是会被经常用到的一个功能。作者把一些indexing的功能（lable or integer）集成在了ix这个方法上。实际中，因为这种label和integer都可以用的方式很方便，于是pandas team设计了loc和ilco来实现label-based和integer-based indexing.

虽然ix indexing依然错在，但是已经过时，不推荐使用。

## 4 Integer Indexes（整数索引）
一些新手再用integer来index的时候，总是会被绊倒。因为这种方法和python用于list和tuple的indexing方法不同。

比如，你不希望下面的代码出现error：